# Setup and Imports

In [1]:
import sys
sys.path.insert(0,'..')  # add project root to PATH

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.python.framework import graph_io
# supress tensorflow depreciation warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import keras
import keras.backend as K
K.set_learning_phase(0)

from src.model import TinyYoloV3

Using TensorFlow backend.


# Export Model

## Helper Functions

In [2]:
def keras_to_pb(model, output_filename, output_node_names):

   """
   This is the function to convert the Keras model to pb.

   Args:
      model: The Keras model.
      output_filename: The output .pb file name.
      output_node_names: The output nodes of the network. If None, then
      the function gets the last layer name as the output node.
   """

   # Get the names of the input and output nodes.
   in_name = model.layers[0].get_output_at(0).name.split(':')[0]

   if output_node_names is None:
       output_node_names = [i.name.split(':')[0] for i in model.outputs]

   sess = keras.backend.get_session()

   # The TensorFlow freeze_graph expects a comma-separated string of output node names.
   output_node_names_tf = ','.join(output_node_names)

   frozen_graph_def = tf.graph_util.convert_variables_to_constants(
       sess,
       sess.graph_def,
       output_node_names)

   sess.close()
   if output_filename is not None and output_filename != "":
       wkdir = ''
       tf.train.write_graph(frozen_graph_def, wkdir, output_filename, as_text=False)

   return frozen_graph_def, in_name, output_node_names

## Load, Optimize and Export 


In [3]:
models_root = '../models/custom_trained_weights'
model_path = f"{models_root}/model_1308_custom_jittering.h5"
pb_model_path = None

model = TinyYoloV3(path=model_path)
# Convert the Keras TinyYolo model to a .pb file
frozen_graph_def, in_tensor_name, out_tensor_names = keras_to_pb(model.model, pb_model_path, None)
print("Input Layer:", in_tensor_name)
print("Output Layers:", out_tensor_names)
print("Optimzing model for TensorRT...")


import tensorflow.contrib.tensorrt as trt

# Create optimized TRT graph
trt_graph = trt.create_inference_graph(
    input_graph_def=frozen_graph_def,
    outputs=out_tensor_names,
    max_batch_size=1,
    max_workspace_size_bytes=1 << 25,
    precision_mode='FP16',
    minimum_segment_size=50
)

graph_io.write_graph(trt_graph, models_root, "trt_graph.pb", as_text=False)

Input Layer: input_1
Output Layers: ['activation_1/Identity', 'activation_2/Identity']
Optimzing model for TensorRT...


'../models/custom_trained_weights/trt_graph.pb'